In [12]:
#!pip install nltk
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [10]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words =[]
classes =[]
documents = []
ignore_letters = ['?','!','.',',']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes :
      classes.append(intent['tag'])
print(documents)

[(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Whats', 'up'], 'greeting'), (['Hey'], 'greeting'), (['greetings'], 'greeting'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['cao'], 'goodbye'), (['see', 'ya'], 'goodbye'), (['What', 'is', 'progamming', '?'], 'programming'), (['What', 'is', 'coding', '?'], 'programming'), (['Tell', 'me', 'about', 'programming'], 'programming'), (['Tell', 'me', 'about', 'coding'], 'programming'), (['What', 'is', 'software', 'development', '?'], 'programming'), (['Where', 'can', 'I', 'learn', 'to', 'code', '?'], 'resource'), (['Best', 'way', 'to', 'learn', 'to', 'code'], 'resource'), (['How', 'can', 'I', 'learn', 'programming'], 'resource'), (['Good', 'programming', 'resources'], 'resource'), (['Can'

In [13]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
print(words)

['Best', 'Can', 'Good', 'Goodbye', 'Have', 'Hello', 'Hey', 'Hi', 'How', 'I', 'Is', 'Leaving', 'See', 'Tell', 'What', 'Whats', 'Where', 'a', 'about', 'am', 'anyone', 'are', 'bye', 'call', 'can', 'cao', 'code', 'coding', 'cya', 'day', 'development', 'good', 'greeting', 'i', 'is', 'later', 'learn', 'me', 'name', 'progamming', 'programming', 'recommend', 'resource', 'see', 'should', 'software', 'there', 'to', 'up', 'way', 'what', 'ya', 'you', 'your']


In [14]:
classes = sorted(set(classes))
pickle.dump(words , open('words.pkl','wb'))
pickle.dump( classes, open('classes.pkl','wb'))


In [15]:
training = []
output_empty = [0] * len(classes)
for document in documents :
  bag =[]
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower())for word in word_patterns]
  for word in words :
    bag.append(1) if word in word_patterns else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(document[1])]=1
  training.append([bag,output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-15-ba6026ea4c1d>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [18]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
sgd = SGD(lr=0.01,momentum=0.9,nesterov = True)
model.compile(loss ='categorical_crossentropy',optimizer=sgd,metrics =['accuracy'])
hist = model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5, verbose = 1)
model.save('chatbot_model.h5',hist)
print('done')

Epoch 1/200
6/6 [==============================] - 1s 4ms/step - loss: 1.6281 - accuracy: 0.2143
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6269 - accuracy: 0.2857
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5229 - accuracy: 0.2857
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4419 - accuracy: 0.4643
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4096 - accuracy: 0.4643
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3707 - accuracy: 0.4286
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.3202 - accuracy: 0.4286
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 1.2544 - accuracy: 0.4643
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.1110 - accuracy: 0.6429
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.0454 - accuracy: 0.5714
Epoch 11/200
6/6 [===========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
!python3 chatbot.py

2023-09-10 11:55:21.861530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
chat pie is running ^_^
hi
1/1 [==============================] - 0s 92ms/step
Hello!
what is your name
1/1 [==============================] - 0s 19ms/step
you can call me chat pie
Traceback (most recent call last):
  File "/content/chatbot.py", line 54, in <module>
    message = input("")
KeyboardInterrupt
^C
